In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso 
using _Data, _Math 

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
function make_colorbar(h, w, header, left_label, right_label, cmap, save_path;rotation=0, label_postion = "top",header_pad = 3,label_color="k")
    fig, ax = subplots(figsize=(w,h), dpi=100)
    font_height=0.45
    r = fig.canvas.get_renderer()

    cb = matplotlib.colorbar.ColorbarBase(ax, cmap=cmap,
                                    norm=matplotlib.colors.Normalize(vmin=0, vmax=1),
                                    orientation="horizontal", ticks=[])
    

    
    dummy = fig.text(-1, -1, "dummy")
    font_height = dummy.get_window_extent(renderer=r).height
    dummy.set_visible(false)


    left_right_labelpad_px = 3
    pad_bottom = 0.0225 / h
    pad_sides = 0.0225 / w
    
    
    cb.set_label(header, labelpad=header_pad, size=7)
    bar_height = (1 - (font_height+header_pad) / (100 * h)) - 2*pad_bottom
    cb.ax.xaxis.set_label_position(label_postion)
    
    bar_height = (1 - font_height / (100 * h)) - 2*pad_bottom
    label_h = pad_bottom + bar_height/2 - 0.03
    
    lbl_left = fig.text(pad_sides, label_h, left_label, horizontalalignment="left", verticalalignment="center_baseline", rotation=rotation,color=label_color)
    bb = lbl_left.get_window_extent(renderer=r)
    left_label_w = (bb.width+left_right_labelpad_px)/(100*w)

    lbl_right = fig.text(1-pad_sides, label_h, right_label, horizontalalignment="right", verticalalignment="center_baseline", rotation=rotation,color=label_color)
    bb = lbl_right.get_window_extent(renderer=r)
    right_label_w = (bb.width+left_right_labelpad_px)/(100*w)
    cb.ax.set_position([pad_sides+left_label_w, pad_bottom, 1-(2*pad_sides+left_label_w+right_label_w), bar_height]) #left, bottom, width, height

    savefig(save_path, format="pdf", bbox_inches = "tight",pad_inches = 0, dpi=250, transparent=true)
end



In [ ]:
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 

In [ ]:
Blues = cm.get_cmap("Blues", 100)
alpha_top = 15
cmaplist = [Blues(i) for i in 1:(Blues.N)]
cmaplist_new = cmaplist[31:end]
Blues_new = mpl_colors.LinearSegmentedColormap.from_list("Blues_new",cmaplist_new,256)

In [ ]:
h=0.2
w=1.2
header = "Overall Occupancy (a.u.)"
left_label = "0"
right_label = "1"
cmap = "Blues"
save_path = "occupancy_colorbar.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path)

In [ ]:
h=0.2
w=1.2
# header = "Place representation# cells"
header = "Place fields (cells)"
left_label = "0"
right_label = "518"
cmap = "hot"
save_path = "map_sum_colorbar.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path)

In [ ]:
colorlist=["k", "r"]
cmp_k_r = mpl_colors.LinearSegmentedColormap.from_list("cmp_k_r", colors=colorlist, N=256)

In [ ]:
h=0.2
w=1.2
header = "Δ\$\\it{F}\$ (z)"
left_label = "0"
right_label = "3"
cmap = cmp_k_r
save_path = "traj_colorbar.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path)

In [ ]:
h=0.2
w=1.2
header = "Δ\$\\it{F}\$ (z)"
left_label = "0"
right_label = "3"
cmap = cmp_k_r
save_path = "traj_colorbar_t.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path;rotation=-90, label_postion = "bottom",header_pad=0)

In [ ]:
h=0.2
w = 0.8
header = "Δ\$\\it{F}\$ (z)"
left_label = "0"
right_label = "3"
cmap = cmp_k_r
save_path = "traj_colorbar_t_short.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path;rotation=-90, label_postion = "bottom",header_pad=0)

In [ ]:
h=0.2
w=1.2
header = "Activity (a.u.)"
left_label = "0"
right_label = "1"
cmap = "jet"
save_path = "map_colorbar.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path)

In [ ]:
h=0.2
w=0.6
header = "Activity (a.u.)"
left_label = "0"
right_label = "1"
cmap = "jet"
save_path = "map_colorbar_t.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path;rotation=-90, label_postion = "bottom",header_pad=2)

In [ ]:
Blues = cm.get_cmap("Blues", 100)
alpha_top = 15
cmaplist = [Blues(i) for i in 1:(Blues.N)]
cmaplist_new = cmaplist[31:end]
Blues_new = mpl_colors.LinearSegmentedColormap.from_list("Blues_new",cmaplist_new,256)

In [ ]:
h=0.2
w=0.6
header = "All"
left_label = "0"
right_label = "1"
cmap = Blues_new
save_path = "corr_dist_matrix_colorbar_1.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path;rotation=-90, label_postion = "bottom",header_pad=0)

In [ ]:
h=0.2
w=0.6
header = "PC"
left_label = "0"
right_label = "1"
cmap = "autumn"
save_path = "corr_dist_matrix_colorbar_2.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path;rotation=-90, label_postion = "bottom",header_pad=0)

In [ ]:
h=0.2
w=1.2
w=1.2
header = "Δ\$\\it{F}\$ (z)"
left_label = "-1"
right_label = "2"
cmap = "hot"
save_path = "activity_colorbar.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path;rotation=-90, label_postion = "bottom",header_pad=0)

In [ ]:
h=0.2
w=1.2
header = "Time (normalized)"
left_label = "0"
right_label = "1"
cmap = "viridis"
save_path = "colorbar_time.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path;rotation=-90, label_postion = "bottom",header_pad=0)

In [ ]:
@pyimport numpy
hot = cm.get_cmap("hot", 100)
alpha_top = 50
cmaplist = [hot(i) for i in 1:(hot.N)]
cmaplist_new = []
length_cmaplist = length(cmaplist)
for (i, color_) in enumerate(cmaplist)
    new_color_ = numpy.copy(color_)
    if i<=alpha_top
        new_color_[4] = (100/alpha_top)*i/length_cmaplist
    end
    append!(cmaplist_new,[new_color_]) 
end
hot_new = mpl_colors.LinearSegmentedColormap.from_list("hot_new",cmaplist_new,100)

In [ ]:
h=0.22
w=0.5
header = ""
left_label = ""
right_label = ""
cmap = cm.get_cmap("hot", 100)
save_path = "distribution_anatomy_colorbar.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path;rotation=-90, label_postion = "top",header_pad=0,label_color="k")

In [ ]:
h=0.2
w=1.0
header = "Occupancy (a.u.)"
left_label = "0"
right_label = "1"
cmap = "Blues"
save_path = "occupancy_colorbar_chuyu.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path;rotation=-90, label_postion = "bottom",header_pad=2)

In [ ]:
h=0.2
w=1.0
header = "Representation (a.u.)"
left_label = "0"
right_label = "1"
cmap = "hot"
save_path = "representation_colorbar_chuyu.pdf"


make_colorbar(h, w, header, left_label, right_label, cmap, save_path;rotation=-90, label_postion = "bottom",header_pad=2)